In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

In [15]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning_v4\Features\FeaturesSC"

## <span style="color:yellow"> **Preparamos el set de entramiento para sacar features** </span> 

In [11]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning_v4\Training Sets"

In [12]:
ins_18_20 = pd.read_csv( loc_ts + "\\ins_18_20.csv" ); ins_18_20["_sc"] = pd.to_datetime(ins_18_20["date"]) - dt.datetime( year = 2019, month = 4, day = 18 ); ins_18_20["_sc"] = ins_18_20["_sc"].dt.total_seconds();
ins_21_23 = pd.read_csv( loc_ts + "\\ins_21_23.csv" ); ins_21_23["_sc"] = pd.to_datetime(ins_21_23["date"]) - dt.datetime( year = 2019, month = 4, day = 21 ); ins_21_23["_sc"] = ins_21_23["_sc"].dt.total_seconds(); 

In [13]:
ins_18_20_frt = ins_18_20.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
ins_21_23_frt = ins_21_23.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )

## <span style="color:yellow"> **Preparamos el ref_hash de cada ventana** </span> 

In [4]:
loc_lb = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning_v4\Labels"

In [8]:
lb_ins_21_23 = pd.read_csv( loc_lb + "\\label_ins_21_23.csv" );
lb_ins_24_26 = pd.read_csv( loc_lb + "\\label_ins_24_26.csv" ); 

In [10]:
rh_trn = lb_ins_21_23[ ["ref_hash"] ]
rh_tst = lb_ins_24_26[ ["ref_hash"] ]

tg_trn = lb_ins_21_23[ ["21_23_sc"] ]
tg_tst = lb_ins_24_26[ ["24_26_sc"] ]

## <span style="color:yellow"> **=======================================================================================================** </span> 

## <span style="color:red"> **Hora de la conversion** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. Por ejemplo que las 00 hs tengan un valor similar a 23

In [88]:
hour_trn = rh_trn.copy()
hour_tst = rh_tst.copy()

hour_frt_ins_trn = ins_18_20_frt[ ["ref_hash" ] ]; hour_frt_ins_trn["hour_ins"] = pd.to_datetime( ins_18_20_frt["date"] ).dt.hour.apply( lambda x: np.sin( (x * np.pi) / 24 ) )
hour_frt_ins_tst = ins_21_23_frt[ ["ref_hash" ] ]; hour_frt_ins_tst["hour_ins"] = pd.to_datetime( ins_21_23_frt["date"] ).dt.hour.apply( lambda x: np.sin( (x * np.pi) / 24 ) ) 

hour_trn = hour_trn.merge( hour_frt_ins_trn, how = "left", on = "ref_hash" )
hour_tst = hour_tst.merge( hour_frt_ins_tst, how = "left", on = "ref_hash" )

hour_trn.to_csv( loc_ftr + "\\hour_ins_trn.csv", index = False )
hour_tst.to_csv( loc_ftr + "\\hour_ins_tst.csv", index = False )

* 129430 nulos de 146578

## <span style="color:red"> **Dia de la conversion** </span> 

Le aplicamos la transformacion sin( (hora * pi)/6 ) para agregarle periodicidad.

In [99]:
week_day_trn = rh_trn.copy()
week_day_tst = rh_tst.copy()

weekday_frt_ins_trn = ins_18_20_frt[ ["ref_hash" ] ]; weekday_frt_ins_trn["week_day"] = pd.to_datetime( ins_18_20_frt["date"] ).dt.weekday.apply( lambda x: np.sin( (x * np.pi) / 24 ) )
weekday_frt_ins_tst = ins_21_23_frt[ ["ref_hash" ] ]; weekday_frt_ins_tst["week_day"] = pd.to_datetime( ins_21_23_frt["date"] ).dt.weekday.apply( lambda x: np.sin( (x * np.pi) / 24 ) ) 

week_day_trn = week_day_trn.merge( weekday_frt_ins_trn, how = "left", on  = "ref_hash" )
week_day_tst = week_day_trn.merge( weekday_frt_ins_tst, how = "left", on  = "ref_hash" )

week_day_trn.to_csv( loc_ftr + "\\week_day_trn.csv", index = False )
week_day_tst.to_csv( loc_ftr + "\\week_day_tst.csv", index = False )

* 129430 nulos de 146578

## <span style="color:red"> **Hora mas popular del usuario en las instalaciones** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. 

In [95]:
main_ihr_trn = rh_trn.copy()
main_ihr_tst = rh_tst.copy()

hour_mode_trn = ins_18_20[["ref_hash"]]; hour_mode_trn["hour"] = pd.to_datetime( ins_18_20["date"] ).dt.hour
hour_mode_tst = ins_21_23[["ref_hash"]]; hour_mode_tst["hour"] = pd.to_datetime( ins_21_23["date"] ).dt.hour

hour_mode_trn = hour_mode_trn.groupby( ["ref_hash", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()
hour_mode_tst = hour_mode_tst.groupby( ["ref_hash", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()

hour_mode_trn = hour_mode_trn.sort_values( ["ref_hash","count"], ascending = False ).drop_duplicates( subset = ["ref_hash"], keep = "first" ).drop("count",axis = 1)
hour_mode_tst = hour_mode_tst.sort_values( ["ref_hash","count"], ascending = False ).drop_duplicates( subset = ["ref_hash"], keep = "first" ).drop("count",axis = 1)

main_ihr_trn = main_ihr_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )
main_ihr_tst = main_ihr_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )

main_ihr_trn["main_hour"] =  main_ihr_trn["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
main_ihr_tst["main_hour"] =  main_ihr_tst["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

main_ihr_trn.to_csv( loc_ftr + "\\main_ihr_trn.csv", index = False )
main_ihr_tst.to_csv( loc_ftr + "\\main_ihr_tst.csv", index = False )

* 129430 nulos de 146578

## <span style="color:green"> **Cantidad de instalaciones por dispositivo en la ventana previa a la conversion** </span> 

In [100]:
cant_ins_trn = rh_trn.copy()
cant_ins_tst = rh_tst.copy()

cant_ins_trn = cant_ins_trn.merge( ins_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" ).fillna( 0 )
cant_ins_tst = cant_ins_tst.merge( ins_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" ).fillna( 0 )

cant_ins_trn.to_csv( loc_ftr + "\\cant_ins_trn.csv", index = False )
cant_ins_tst.to_csv( loc_ftr + "\\cant_ins_tst.csv", index = False )

## <span style="color:red"> **IP del dispositivo al realizar la primera instalacion** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> 

In [108]:
ip_encod_trn = rh_trn.copy()
ip_encod_tst = rh_tst.copy()

ip_ins_trn = ins_18_20_frt[ ["ref_hash","ip_address"] ];
ip_ins_tst = ins_21_23_frt[ ["ref_hash","ip_address"] ];

appces_trn_ins = ins_18_20["ip_address"].value_counts().reset_index().rename( columns = {"index":"ip_address", "ip_address":"appearances"} )
appces_tst_ins = ins_21_23["ip_address"].value_counts().reset_index().rename( columns = {"index":"ip_address", "ip_address":"appearances"} )

appces_trn_ins["ip_address_mean"] = appces_trn_ins["appearances"] / len( ins_18_20 ); appces_trn_ins.drop( "appearances", axis = 1, inplace = True )
appces_tst_ins["ip_address_mean"] = appces_tst_ins["appearances"] / len( ins_21_23 ); appces_tst_ins.drop( "appearances", axis = 1, inplace = True )

ip_ins_trn = ip_ins_trn.merge( appces_trn_ins, how = "left", on = "ip_address" )
ip_ins_tst = ip_ins_tst.merge( appces_tst_ins, how = "left", on = "ip_address" )

ip_encod_trn = ip_encod_trn.merge( ip_ins_trn, how = "left", on = "ref_hash" ).drop( "ip_address", axis = 1 )
ip_encod_tst = ip_encod_tst.merge( ip_ins_tst, how = "left", on = "ref_hash" ).drop( "ip_address", axis = 1 )

ip_encod_trn.to_csv( loc_ftr + "\\ip_encod_trn.csv", index = False )
ip_encod_tst.to_csv( loc_ftr + "\\ip_encod_tst.csv", index = False )

* 129430 nulos de 146578

## <span style="color:green"> **WiFi en la instalacion** </span>

#### <span style="color:orange"> **Mean Encoding** </span> 

In [47]:
wifi_ins_trn = rh_trn.copy()
wifi_ins_tst = rh_tst.copy()

sc = ins_21_23_frt.loc[ ins_21_23_frt["ref_hash"].isin( rh_trn["ref_hash"] ), ["ref_hash","_sc"] ]

wf_trn_sc = ins_18_20_frt[ ["ref_hash","wifi"] ].merge( sc, how = "left", on = "ref_hash" ); 

X_wf = wf_trn_sc[ ["wifi"] ].astype(str)
Y_wf = wf_trn_sc[ ["_sc"] ]

te = ce.TargetEncoder(smoothing=True); te.fit( X_wf, Y_wf );

wifi_ins_trn["wifi"] = te.transform( ins_18_20_frt[ ["wifi"] ].astype( str ) );
wifi_ins_tst["wifi"] = te.transform( ins_21_23_frt[ ["wifi"] ].astype( str ) ); 

wifi_ins_trn.to_csv( loc_ftr + "\\wifi_ins_trn.csv", index = False )
wifi_ins_tst.to_csv( loc_ftr + "\\wifi_ins_tst.csv", index = False )

#### <span style="color:orange"> **OneHotEncoding** </span> (Usando el SC para el promedio) - <span style="color:red"> **No Mejora este tipo de codificacion** </span>

In [21]:
wifi_ins_trn = rh_trn.copy()
wifi_ins_tst = rh_tst.copy()

wf_ohe_trn = pd.get_dummies( ins_18_20_frt[ ["wifi"] ] )
wf_ohe_tst = pd.get_dummies( ins_21_23_frt[ ["wifi"] ] )

wifi_ins_trn = pd.concat( [wifi_ins_trn, wf_ohe_trn], axis = 1 )
wifi_ins_tst = pd.concat( [wifi_ins_tst, wf_ohe_tst], axis = 1 )

wifi_ins_trn.to_csv( loc_ftr + "\\wifi_ins_trn.csv", index = False )
wifi_ins_tst.to_csv( loc_ftr + "\\wifi_ins_tst.csv", index = False )

## <span style="color:green"> **Aplicacion mas instalada por el usuario** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la app principal de algun dispositivo)

In [55]:
main_app_trn = rh_trn.copy()
main_app_tst = rh_tst.copy()

app_mas_ins_trn = ins_18_20.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()
app_mas_ins_tst = ins_21_23.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()

app_mas_ins_trn = app_mas_ins_trn.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_trn["cant_ins"] )
app_mas_ins_tst = app_mas_ins_tst.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_tst["cant_ins"] ) 

main_app_trn = main_app_trn.merge( app_mas_ins_trn, how = "left", on = "ref_hash" )
main_app_tst = main_app_tst.merge( app_mas_ins_tst, how = "left", on = "ref_hash" )

main_app_trn["to_count"] = 1; main_app_trn["application_id"] = main_app_trn[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_trn); del(main_app_trn["to_count"])
main_app_tst["to_count"] = 1; main_app_tst["application_id"] = main_app_tst[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_tst); del(main_app_tst["to_count"])

main_app_trn.fillna( main_app_trn.isnull().sum()/len(main_app_trn) )
main_app_tst.fillna( main_app_tst.isnull().sum()/len(main_app_tst) )

main_app_trn.to_csv( loc_ftr + "\\main_app_trn.csv", index = False )
main_app_tst.to_csv( loc_ftr + "\\main_app_tst.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones implicitas por dispositivo** </span> 

In [56]:
cins_imp_trn = rh_trn.copy()
cins_imp_tst = rh_tst.copy()

cins_imp_trn["cant_imp"] = ins_18_20["implicit"].apply( lambda x: 1 if x else 0 ); cins_imp_trn = cins_imp_trn.groupby( "ref_hash" ).agg( {"cant_imp":"sum"} ).reset_index()
cins_imp_tst["cant_imp"] = ins_21_23["implicit"].apply( lambda x: 1 if x else 0 ); cins_imp_tst = cins_imp_tst.groupby( "ref_hash" ).agg( {"cant_imp":"sum"} ).reset_index()

cins_imp_trn.to_csv( loc_ftr + "\\cins_imp_trn.csv", index = False )
cins_imp_tst.to_csv( loc_ftr + "\\cins_imp_tst.csv", index = False )

## <span style="color:green"> **Tipo de la primera instalacion (implicita/no implicita)** </span> 

#### <span style="color:orange"> **OneHotEncoding** </span> 

In [66]:
type_ins_trn = rh_trn.copy()
type_ins_tst = rh_tst.copy()

type_ins_ohe_trn = pd.get_dummies( ins_18_20_frt[ ["implicit"] ].astype( "category" ) )
type_ins_ohe_tst = pd.get_dummies( ins_21_23_frt[ ["implicit"] ].astype( "category" ) )

rh_18_20 = ins_18_20_frt[ ["ref_hash"] ]

type_ins_by_ref_hash_18_20 = pd.concat( [ins_18_20_frt[ ["ref_hash"] ], type_ins_ohe_trn], axis = 1 )
type_ins_by_ref_hash_21_23 = pd.concat( [ins_21_23_frt[ ["ref_hash"] ], type_ins_ohe_tst], axis = 1 )
                            
type_ins_trn = type_ins_trn.merge( type_ins_by_ref_hash_18_20, how = "left", on = "ref_hash" )
type_ins_tst = type_ins_tst.merge( type_ins_by_ref_hash_21_23, how = "left", on = "ref_hash" )
    
type_ins_trn.to_csv( loc_ftr + "\\type_ins_trn.csv", index = False )
type_ins_tst.to_csv( loc_ftr + "\\type_ins_tst.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones atribuidas por dispositivo** </span> 

In [59]:
cins_atr_trn = rh_trn.copy()
cins_atr_tst = rh_tst.copy()

cins_atr_trn["cant_atr"] = ins_18_20["attributed"].apply( lambda x: 1 if x else 0 ); cins_atr_trn = cins_atr_trn.groupby( "ref_hash" ).agg( {"cant_atr":"sum"} ).reset_index()
cins_atr_tst["cant_atr"] = ins_21_23["attributed"].apply( lambda x: 1 if x else 0 ); cins_atr_tst = cins_atr_tst.groupby( "ref_hash" ).agg( {"cant_atr":"sum"} ).reset_index()

cins_atr_trn.to_csv( loc_ftr + "\\cins_atr_trn.csv", index = False )
cins_atr_tst.to_csv( loc_ftr + "\\cins_atr_tst.csv", index = False )